In [1]:
# ALV: modified from https://github.com/lauringlab/Host_level_IAV_evolution/blob/master/results/Investigating%20mixed%20infections.ipynb
# Purpose: use FastTree to make a phylogenetic tree of IBV concatenated consensus sequences.
# Modify to make an annotated tree. This one will just have sequencing IDs.

import numpy as np
import pandas as pd
import copy 
from matplotlib import pyplot as plt
import os
import tempfile
import sys
import subprocess
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio.Alphabet import IUPAC
from Bio import Phylo
from ast import literal_eval

import re

In [17]:
def trim_to_coding(fasta):
    """This funciton trims a sample fasta sequence to the reading frame defined in a separate fasta file
    First the funciton takes in the SPECID and looks up where the sample fasta file will be using the meta data
    available in the meta argument. For each sequence in the reference the function looks for the same sequence name
    in the sample fasta. It then alings this and trims the gaps so we are left with just the OR."""
    
    #samp_meta = get_meta(SPECID,meta)
    
    regions = "Run_2445/data/reference/YAM_PlasmidControl_OR.fa"
    #regions = "../fasta_for_fasttree/VIC_PlasmidControl_OR_mod.fa"
    
    coding = ReadFASTA(regions)
    # cycle through fasta 
    OR=[]
    for gene in fasta:
        seg_id=gene.id  
        gene.description = "this is the test sample " + gene.description
        for code in coding:
            code_id = code.id
            if seg_id ==code_id:
                #print('working with %s' %seg_id) #and seg_id=="NR":
                code_gene=Align([code,gene],"/sw/lsa/centos7/muscle/3.8.31/bin/")
                code_gene_trimmed = StripGapsToFirstSequence(code_gene)
                OR.append(code_gene_trimmed)
        
    return(OR)

In [3]:
sys.path.append("/home/avalesan/variant_pipeline/scripts")
from fasta_functions import StripGapsToFirstSequence, Align

In [4]:
def ReadFASTA(fastafile):
    """Reads sequences from a FASTA file.

    'fastafile' should specify the name of a FASTA file.

    This function reads all sequences from the FASTA file.  It returns the
        list 'headers_seqs'.  This list is composed of a seq_record objects.
    """
    seqs =[]
    header = None
    for seq_record in SeqIO.parse(fastafile, "fasta"):
        seq_record.seq.alphabet=IUPAC.unambiguous_dna
        seqs.append(seq_record)

    return seqs

In [5]:
def concat_seqs(sequences):
    """This funciton concatenates sequences present in a dictionary of sequences."""
    concat_seq={}
    for key in sequences:
        str_seq = "".join([str(seq_rec.seq) for seq_rec in sequences[key]])
        concat_seq[key]= [SeqRecord( Seq(str_seq),id="All",name=key)]# needs to be list to match function above
    return(concat_seq)

In [18]:
def get_seq(specid_list):
    """This function takes a list of SPECIDs and data frame with meta data for the samples. It is similar to the 
    function above but it does not add isnv to the sequences. It only returns the consensus sequences.
    For each SPECID we get the meta data in dictionary form,g et the consensus fasta file, 
    Trim the new sequences to the coding regions and return a dictionary of the results indexed by SPECID."""
    sequences={}
    for specid in specid_list:
        print("working on specid: ")
        print(specid)
        fa = "consensus_fa/YAM/" + specid + ".removed.parsed.fasta"
        seq = ReadFASTA(fa)
    
        for seg in seq:
            seg.name=specid
        
        seg_coding = trim_to_coding(seq)
        sequences[specid] = seg_coding
    return(sequences)

In [6]:
def make_alignments(seg,sequences,out_dir,out_file):
    """ This function aligns the sequences provided using muscle. The input is a list of dictionaries of 
    sequences. The segment of interest is provided as the seg argument.
    """
    
    segment = []
    for sample in sequences:
        for chrom in sequences[sample]:
            if chrom.id==seg:
                seg_copy = copy.deepcopy(chrom)
                seg_copy.id = seg_copy.name # original line
                #seg_copy.id = sample # modified for running by segment. Should still work for all?
                segment.append(seg_copy)
    
    muscle_progpath = "/sw/lsa/centos7/muscle/3.8.31/bin/"
    muscle_exe = os.path.abspath("%s/muscle" % muscle_progpath) # the executable
    
    
    currdir = os.getcwd()
    # make directory if it doesn't exist.
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    
    infile_fa = "%s/in.fasta" % out_dir # input file
    align_fa = "%s/%s" % (out_dir,out_file) # output file
    
   
    print("Writing %d sequences to file" % len(segment))
    
    SeqIO.write(segment, infile_fa, "fasta") # write sequences to the input file
    
    print("%s\n %s -in %s -out %s" % ("Making alignment",muscle_exe, infile_fa, align_fa))
    
    p = subprocess.Popen("%s -in %s -out %s" % (muscle_exe, infile_fa, align_fa), shell = True, stdout = subprocess.PIPE, stderr = subprocess.PIPE) # run MUSCLE
    (output, errors) = p.communicate()

In [7]:
def make_tree(in_fasta,tree_file):
    # Make the tree
    """This function uses fasttree to make a ML tree of the sequences present in an aligned fasta file"""
    tree_progpath = "/sw/med/centos7/fasttree/2.1.9/bin/"
    tree_exe = os.path.abspath("%s/FastTree" % tree_progpath) # the executable
    #tree_file = "%s/tree.file" % tempdir
    print(" %s \n %s -nt %s > %s" % ("Making tree:",tree_exe, in_fasta,tree_file))
    
    t = subprocess.Popen("%s -nt %s > %s" % (tree_exe, in_fasta,tree_file), shell = True, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    (output, errors) = t.communicate()

In [8]:
# Order to modify: trim_to_coding, get_seq; make_alignments, make_tree
# Then run get_seq, concat_seq, make_alignments, and make_tree, in that order
os.getcwd()

'/scratch/alauring_fluxm/avalesan/IBV_HIVE'

In [19]:
# Will need a list of all the sequence ids to process.
#names = pd.read_csv("../fasta_for_fasttree/names.txt")
names = pd.read_csv("YAM_IDs.csv", dtype={'SeqSampleNumber1': str})
specids = names['SeqSampleNumber1']
#print specids
IBV_seq = get_seq(specids)
print "done!"

working on specid: 
114234
working on specid: 
114235
working on specid: 
114286
working on specid: 
114285
working on specid: 
114237
working on specid: 
114240
working on specid: 
114284
working on specid: 
114283
working on specid: 
114242
working on specid: 
114243
working on specid: 
114244
working on specid: 
114245
working on specid: 
114246
working on specid: 
114278
working on specid: 
114249
working on specid: 
114277
working on specid: 
114276
working on specid: 
114250
working on specid: 
114251
working on specid: 
114217
working on specid: 
114261
working on specid: 
114218
working on specid: 
114188
working on specid: 
114219
working on specid: 
114211
working on specid: 
114209
working on specid: 
114213
working on specid: 
114216
working on specid: 
114214
working on specid: 
114207
working on specid: 
114210
working on specid: 
114223
working on specid: 
114232
working on specid: 
114266
working on specid: 
114225
working on specid: 
114229
working on specid: 
114233
w

In [20]:
IBV_concat_seq = concat_seqs(IBV_seq)
print "done concatenating"

done concatenating


In [21]:
make_alignments(seg="All",sequences=IBV_concat_seq,out_dir=".",out_file="IBV_YAM_aln.fa")
print "done with make_alignments"

Writing 62 sequences to file
Making alignment
 /sw/lsa/centos7/muscle/3.8.31/bin/muscle -in ./in.fasta -out ./IBV_YAM_aln.fa
done with make_alignments


In [101]:
make_tree("../fasta_for_fasttree/IBV_coding_aln_NS.fa","../fasta_for_fasttree/IBV_coding_aln_NS.tree")
print "done making tree"

 Making tree: 
 /sw/med/centos7/fasttree/2.1.9/bin/FastTree -nt ../fasta_for_fasttree/IBV_coding_aln_NS.fa > ../fasta_for_fasttree/IBV_coding_aln_NS.tree
done making tree


In [99]:
IBV_seq

{'114175': [SeqRecord(seq=Seq('ATGAATATAAATCCTTATTTCCTCTTCATAGATGTGCCCATACAGGCAGCAATT...TAA', IUPACUnambiguousDNA()), id='PB1', name='PB1', description='PB1 this is the test sample PB1', dbxrefs=[]),
  SeqRecord(seq=Seq('ATGACATTGGCCAAAATTGAATTGTTGAAACAACTGCTAAGGGACAATGAAGCC...TAA', IUPACUnambiguousDNA()), id='PB2', name='PB2', description='PB2 this is the test sample PB2', dbxrefs=[]),
  SeqRecord(seq=Seq('ATGGATACTTTTATTACAAGAAACTTCCAGACTACAATAATACAAAAGGCCAAA...TAA', IUPACUnambiguousDNA()), id='PA', name='PA', description='PA this is the test sample PA', dbxrefs=[]),
  SeqRecord(seq=Seq('ATGAAGGCAATAATTGTACTACTCATGGTAGTAACATCCAATGCAGATCGAATC...TAA', IUPACUnambiguousDNA()), id='HA', name='HA', description='HA this is the test sample HA', dbxrefs=[]),
  SeqRecord(seq=Seq('ATGTCCAACATGGATATTGATGGTATAAACACTGGGACAATTGACAAAACACCG...TAA', IUPACUnambiguousDNA()), id='NP', name='NP', description='NP this is the test sample NP', dbxrefs=[]),
  SeqRecord(seq=Seq('ATGCTACCTTCAACTATACAAATGTTAACCC